<a href="https://colab.research.google.com/github/yahyahus10/N_advice_chatbot/blob/main/Nutritional_guidance_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Chroma is a database for building AI applications with embeddings.
!pip install langchain-chroma

In [ ]:
!pip install langchain_community
!pip install langchain_text_splitters

In [6]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,)
from langchain_text_splitters import CharacterTextSplitter

In [ ]:
!pip install pypdf

In [12]:
from pypdf import PdfReader

In [23]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [24]:
def load_pdf(data):
    loader = PyPDFDirectoryLoader(data, glob="*.pdf")

    documents=loader.load()

    return documents

In [25]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [27]:
document=load_pdf("data/")

In [ ]:
document

In [39]:
def text_split(extracted_data):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=25)
  text_chunks = text_splitter.split_documents(extracted_data)
  return text_chunks

In [40]:
docs=text_split(document)

In [41]:
print("length of my chunk", len(docs))

length of my chunk 834


In [ ]:
!pip install sentence-transformers==2.2.2

In [ ]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [42]:
#load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

In [45]:
#query it
query = "What are CVD"
result= db.similarity_search(query)

In [46]:
result

[Document(page_content='CVD, based mainly on case-control studies [63,64] . However, in the last  few years there are \naccumulated data suggesting no harm [65–67], and ev en a protective associa tion between moderate', metadata={'page': 11, 'source': 'data/Nutritional advice for Cardiovascular Disease Prevention.pdf'}),
 Document(page_content='CVD, based mainly on case-control studies [63,64] . However, in the last  few years there are \naccumulated data suggesting no harm [65–67], and ev en a protective associa tion between moderate', metadata={'page': 11, 'source': 'data/Nutritional advice for Cardiovascular Disease Prevention.pdf'}),
 Document(page_content='and vitamins (riboflavin and vitamin B-12) that can ex ert beneficial effects on CVD. On the other hand,', metadata={'page': 8, 'source': 'data/Nutritional advice for Cardiovascular Disease Prevention.pdf'}),
 Document(page_content='and vitamins (riboflavin and vitamin B-12) that can ex ert beneficial effects on CVD. On the othe

In [37]:
# save to disk
#load  it into Chroma
db2 = Chroma.from_documents(docs, embedding_function, persist_directory="./chroma_db")

In [ ]:
!pip install langchain

In [61]:
!pip install ctransformers==0.2.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 8.7 MB/s eta 0:00:00


In [62]:
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [53]:
promt_template="""
Use the following pieces of information to answer the user's question. If you don't know the answer, just say you don't know, don't try to make up an answer.

Context:{context}
Question:{question}

Only return the helpful answer below and nothing else.
Helpful answer:"""

In [55]:
PROMPT=PromptTemplate(template=promt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt":PROMPT}

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [58]:
model_path = '/content/drive/MyDrive/llama-2-7b-chat.ggmlv3.q4_0.bin'

In [63]:
llm = CTransformers(
    model="/content/drive/MyDrive/llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type="llama",
    config={
        'max_new_tokens': 512,
        'temperature': 0.8  # Corrected to proper key-value format
    }
)

In [64]:
from langchain.chains import RetrievalQA


In [68]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={'k': 2}),#k=2 :Will give two relevant answer
    return_source_documents=True,  # Corrected spelling
    chain_type_kwargs=chain_type_kwargs
)

In [69]:
while True:
  user_input=input(f"Input Prompt:")
  result=qa({"query": user_input})
  print("Response :", result["result"])


Input Prompt:What are the effects of alcoholics drinks on CVD ?


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response :  The evidence for an association between alcohol consumption and cardiovascular disease (CVD) comes from both short-term interventional studies on the effects of alcohol on risk factors as well as long-term observational mortality studies.


KeyboardInterrupt: Interrupted by user

In [70]:
pip install flask
